# 4.1 Task 1: Safe Grid World Environment Setup
Begin by implementing the safe grid-world environment from scratch. The environment must support standard gym-like methods: reset() returns the initial state, step(action) executes an action and returns the next state, reward, and done flag, and render() visualises the current state. You must use the following exact configuration to ensure consistency across all student submissions:

## Environment Configuration:
• Grid size: 10 × 10
• Start position: (0, 0) (top-left corner)
• Goal position: (9, 9) (bottom-right cor-ner)
• Hazard cells (15 total): (0, 3), (1, 1), (1, 7), (2, 4), (2, 8), (3, 2), (3, 6), (4, 5), (5, 3), (5, 8), (6, 1), (6, 6), (7, 4), (8, 2), (8, 7)
• Walls: Grid boundaries only (no internal walls)
• Action space: UP, DOWN, LEFT, RIGHT (4 discrete actions)
• Reward structure: +10 for reaching goal, −0.1 per time step (step penalty), −10 for entering hazard (episode terminates immediatel

## Figure 1: Safe Grid World environment config-
uration showing the 10×10 grid with start position (S, blue, top-left), goal position (G, green, bottom-right), and 15 hazard cells (red) strategically distributed throughout the grid. Safe cells are shown in white

## 
    This configuration strategically distributes hazards throughout the grid to test safety shield generalisation and safety-aware navigation. The distributed placement prevents the agent from learning simple avoidance rules (such as “avoid certain columns”) and instead requires learning the true underlying safety constraints whilst maintaining multiple solvable paths to the goal.
    Implement collision detection for walls (agent stays in place if attempting to move out of bounds) and hazard detection (episode terminates immediately upon entering a hazard cell). Random Start Position (Required Feature). Your environment must support two start position modes controlled by a boolean parameter in the environment constructor. In Fixed Start Mode (default), every episode begins at position (0, 0), matching the standard configuration above. In Random Start Mode, each episode begins at a randomly sampled safe position (any cell that is not a hazard and not the goal), ensuring broader state-space exploration during training